# Python Testing Tutorial

Welcome to the Python Testing Tutorial notebook. This will be the resource we use to both learn, and test what we learned. This notebook is designed to be loaded to a Jupyter Lab instance that has `pytest` and `ipython_pytest` installed. In a new virtual environment, do

```
$ pip install pytest ipython_pytest jupyterlab
```

When this is done, launch Jupyter

```
$ jupyter lab
```

Click on the upload icon, and upload this notebook.

The next step will be to load the `ipython_test` Jupyter extension:

In [ ]:
%load_ext ipython_pytest

There should not be any output from this step. If an error occured saying it is not installed, make sure the virtual environment has `ipython_test` installed.

Next, we will run one test, just to see what failure looks like:

In [ ]:
%%pytest

def test_something():
    assert [1] == [2]

If your test didn't run, or you did not get a failure that looks like

```
    def test_something():
>       assert [1] == [2]
E       assert [1] == [2]
E         At index 0 diff: 1 != 2
E         Use -v to get the full diff

_ipytesttmp.py:3: AssertionError
```

Then something might not be installed and configured properly. Check again that `pytest` and `ipython_pytest` are properly installed in your virtual environment.

The mechanics of running tests differ between environments. Usually, you will be running the `pytest` command-line -- but quite often, not directly. You might be using a `tox` runner, run the tests in a Continuous Integration environment, or maybe inside a Docker container. Regardless, most of the effort will be spent writing tests and looking at test failures -- and this is going to be what this tutorial will cover.

Pytest uses the Python keyword `assert` in order to check conditions in tests. It internally processes this statement to help failures look nicer. We already saw that it does a checks the difference with a list of one element. Let's see a few more examples:

In [ ]:
%%pytest

def test_missing():
    assert [1] == []
    
def test_extra():
    assert [1] == [1, 2]
    
def test_different():
    assert [1, 2, 3] == [1, 4, 3]

All tests failed here -- but read the output carefully to see *how* they failed. When writing tests for new code, much of your work will be analyzing test failures to understand how they failed, and whether the test code or product code is broken. If the tests are doing their job to prevent regressions, much of your work will be looking at test failures to understand whether you need to fix the code or modify an out-of-date test.

The most interesting thing about any test framework is how test failures look.

Let's see how `pytest` handles other equality failures:

In [ ]:
%%pytest

def test_string():
    assert "hello\nworld" == "goodbye\nworld"

For strings, `pytest` will do a line-by-line diff to highlight differences. However, it will not do inside-line-diff

In [ ]:
%%pytest

def test_set():
    assert set([1,2]) == set([1,3])

For sets, it will check for spurious elements on both sides.

The more code we put on the assertion line, the more detailed the output will be.

In [ ]:
%%pytest

def add(x, y):
    return x+y
    
def test_add():
    assert add(1, 2) == 4

But sometimes equality is too strict: after all many functions we write, it's hard to know what the output will be equal to.

In [ ]:
%%pytest

import random

def test_random():
    assert random.uniform(0, 1) > 2

We can also check for set membership:

In [ ]:
%%pytest

import random

def test_choice():
    assert random.choice([1, 2, 3]) in [4, 5]

Testing edge-cases of code is particularly important. Specifically, we might want to test whether the code raises the right exception.

In [ ]:
%%pytest

import pytest

def test_exception():
    with pytest.raises(ValueError):
        1/0

OK, now it is time to fix some tests. In each cell, only change the line that says "fix this line"

In [ ]:
%%pytest

def add(a, b):
    return a # fix this line

def test_add():
    asserrt add(1, 2) == 3

In [ ]:
%%pytest

def append(a, b):
    pass # fix this line

def test_add():
    a = [1, 2, 3]
    append(a, 4)
    assert a == [1, 2, 3, 4]

In [ ]:
%%pytest

def safe_remove(a, b):
    pass # fix this line

def test_safe_remove_no():
    things = {1: "yes", 2: "no"}
    safe_remove(things, 3)
    assert 1 in things

def test_safe_remove_yes():
    things = {1: "yes", 2: "no"}
    safe_remove(things, 2)
    assert 2 not in things

In [ ]:
%%pytest

def get_min_max(a, b):
    return a, b # fix this line

def test_min_max_high():
    a, b = get_min_max(2, 1)
    assert set([a, b]) == set([1, 2])
    assert a < b

def test_min_max_low():
    a, b = get_min_max(1, 2)
    assert set([a, b]) == set([1, 2])
    assert a < b

In [ ]:
%%pytest

import pytest

def test_no_element():
    thing = {} # fix this line
    with pytest.raises(IndexError):
        thing[0]

## Recap

* Tests check whether things are true
* pytest uses `assert`
* Put calculations on the `assert` line to get more information

## Mocking

Sometimes, using real objects is hard, ill-advised, or complicated.

For example, a `requests.Session` connects to real websites:
using it in your unittests invites a...lot...of problems.

In [ ]:
from unittest import mock

"Mocks" are a unittest concept: they produce objects that are substitutes for the real ones.
There's a whole cottage industry that will explain that "mock", "fake", and "stub" are all subtly different.
We'll use all of those interchangably.

In [ ]:
regular = mock.Mock()

def do_something(o):
    return o.something(5)

do_something(regular)

Mocks have "all the methods". The methods will usually return another Mock.

In [ ]:
# Read words from a pathlib.Path
import contextlib

def words_from_path(path_obj):
    with contextlib.closing(path_obj.open()) as fpin:
        result = fpin.read()
    return result.split()

words_from_path(regular)

What if we want it to return a "real" string?

In [ ]:
%%pytest

import contextlib

def words_from_path(path_obj):
    with contextlib.closing(path_obj.open()) as fpin:
        result = fpin.read()
    return result.split()

from unittest import mock

my_path = mock.Mock()
my_path.open.return_value.read.return_value = """\
In winter, when the fields are white,
I sing this song for your delight.
In spring, when woods are getting green,
I'll try and tell you what I mean.
In summer, when the days are long,
Perhaps you'll understand the song.
In autumn, when the leaves are brown,
Take pen and ink, and write it down.
"""

def test_jabberwocky():
    result = words_from_path(my_path)
    assert "winter" in result